In [ ]:
!pip install pyngrok
!pip install faster-whisper
!ngrok config add-authtoken <your token>
!pip install fastapi
!git clone https://gist.github.com/gokul6350/4793092d5943ea38879df861bcaecd09.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.3/192.3 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 13.6 MB/s eta 0:00:00
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 943.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
!gcloud auth application-default login \
  --no-browser --client-id-file /content/client_secret.json \
  --scopes https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning,https://www.googleapis.com/auth/generative-language.retriever

You are authorizing client libraries without access to a web browser. Please run the following command on a machine with a web browser and copy its output back here. Make sure the installed gcloud version is 372.0.0 or newer.

gcloud auth application-default login --remote-bootstrap="https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=831752801621-nc9rdedhutk7btnb33cvqohsh3rrglmm.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgenerative-language.tuning+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgenerative-language.retriever&state=ZoCkGVb9DtUGGbbd0ryStl3GPUuKPS&access_type=offline&code_challenge=txy2ypIb1lVHQAcehy6smyazajju7djOaAAQVyOnbXQ&code_challenge_method=S256&token_usage=remote"


Enter the output of the above command: https://localhost:8085/?state=ZoCkGVb9DtUGGbbd0ryStl3GPUuKPS&code=4/0AdLIrYcOhi8_8U4VvZMez1L3xlVvriFI7LpiDmLgH0I4JPlGLCWmZXkLVqBj7W8iqTb-DA&scope=https://www.googl

### TESTING LLM

In [ ]:

import google.generativeai as genai
generation_config = {
    "temperature": 0.7,
    "top_p": 1,
    "top_k": 0,
    "max_output_tokens": 8192,
}

safety_settings = []

model1 = genai.GenerativeModel(model_name="tunedModels/dataset-reb3qi4cuvqu",
                               generation_config=generation_config,
                               safety_settings=safety_settings)


def gen_ai(prompt):

    with open('4793092d5943ea38879df861bcaecd09/prompt.txt', 'r') as file:
      content = file.read().splitlines()


    frompt_parts = [line.format(prompt=prompt).strip() for line in content]


    response = model1.generate_content(frompt_parts)
    print(f">>>{response.text}")

    return response.text
print(gen_ai("hello"))
print(gen_ai("whats you name ?"))

>>>_hello
_hello


In [ ]:
import os
import threading
from fastapi import FastAPI, File, UploadFile, HTTPException ,Form
from fastapi.responses import JSONResponse
from pyngrok import ngrok
from faster_whisper import WhisperModel
import google.generativeai as genai
import psutil
from datetime import datetime
import uuid
import uvicorn

model_size = "large-v3"
model = WhisperModel(model_size, device="cuda", compute_type="float16")
app = FastAPI()
port = "5000"

generation_config = {
    "temperature": 0.7,
    "top_p": 1,
    "top_k": 0,
    "max_output_tokens": 8192,
}

safety_settings = []

model1 = genai.GenerativeModel(model_name="tunedModels/dataset-reb3qi4cuvqu",
                               generation_config=generation_config,
                               safety_settings=safety_settings)

def gen_ai(prompt):

    with open('4793092d5943ea38879df861bcaecd09/prompt.txt', 'r') as file:
      content = file.read().splitlines()


    frompt_parts = [line.format(prompt=prompt).strip() for line in content]


    response = model1.generate_content(frompt_parts)
    print(f">>>{response.text}")

    return response.text

def is_port_in_use(port):
    for proc in psutil.process_iter():
        try:
            pinfo = proc.as_dict(attrs=['pid', 'name', 'cmdline'])
            if 'python' in pinfo['name'] and 'uvicorn' in ' '.join(pinfo['cmdline']) and port in ' '.join(pinfo['cmdline']):
                return True
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass
    return False

def find_available_port(start_port):
    port = start_port
    while is_port_in_use(str(port)):
        port += 1
    return port

def stop_existing_uvicorn_server(port):
    for proc in psutil.process_iter():
        try:
            pinfo = proc.as_dict(attrs=['pid', 'name', 'cmdline'])
            if 'python' in pinfo['name'] and 'uvicorn' in ' '.join(pinfo['cmdline']) and port in ' '.join(pinfo['cmdline']):
                print(f"Stopping existing Uvicorn server (PID: {pinfo['pid']})...")
                proc.terminate()
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass

stop_existing_uvicorn_server(port)
new_port = find_available_port(int(port))
print(f"Using port: {new_port}")

public_url = ngrok.connect(new_port).public_url
print(f" * ngrok tunnel {public_url} -> http://127.0.0.1:{new_port}")
app.state.base_url = public_url

@app.get("/")
async def index():
    return {"message": "Hello from Colab!"}

def generate_unique_filename():
    unique_id = uuid.uuid4().hex
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    return f"{timestamp}_{unique_id}.wav"

@app.post("/transcribe")
async def transcribe_audio(file: UploadFile = File(...), text: str = Form(...)):
    if not file:
        raise HTTPException(status_code=400, detail="No file part")
    if file.filename == '':
        raise HTTPException(status_code=400, detail="No selected file")

    print(f"FILE received: {file.filename}")
    print(f"TEXT received: {text}")

    unique_filename = generate_unique_filename()
    file_path = f"/content/{unique_filename}"

    with open(file_path, "wb") as buffer:
        buffer.write(file.file.read())

    print(f"FILE SAVED: {unique_filename}")

    segments, info = model.transcribe(file_path, beam_size=5, vad_filter=True, vad_parameters=dict(min_silence_duration_ms=500))

    print(f"Audio loaded: {unique_filename}")

    for segment in segments:
        print(f"[{segment.start:.2f}s -> {segment.end:.2f}s] {segment.text}")
    promt=f"you can see {text}. {segment.text}"
    print(promt)
    rep = gen_ai(promt)
    return JSONResponse(content={'reply': rep, 'text': segment.text, 'Request_ID': unique_filename})

def start_uvicorn():
    uvicorn.run(app, host="0.0.0.0", port=new_port, log_level="info")

if __name__ == "__main__":
    thread = threading.Thread(target=start_uvicorn, daemon=True)
    thread.start()

    print("$$$$$$$$$$$$$$$$$$$$$$$$$")
    input("Press Enter to stop the server...")

    os._exit(0)


Using port: 5000
 * ngrok tunnel https://9f80-34-27-36-224.ngrok-free.app -> http://127.0.0.1:5000
$$$$$$$$$$$$$$$$$$$$$$$$$


INFO:     Started server process [220]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 5000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


FILE received: recorded_audio.wav
TEXT received: battery,glue,lead,object_1
FILE SAVED: 20240527131848_790160b20fdc4e3cb49e2058f12d9bdd.wav
Audio loaded: 20240527131848_790160b20fdc4e3cb49e2058f12d9bdd.wav
[0.00s -> 1.68s]  Pick up battery.
you can see battery,glue,lead,object_1.  Pick up battery.
>>>_pickup(battery) # picking up the battery
INFO:     2405:201:e00a:7103:6a35:286:baae:3cd3:0 - "POST /transcribe HTTP/1.1" 200 OK
FILE received: recorded_audio.wav
TEXT received: battery,glue,lead,object_1
FILE SAVED: 20240527131906_708cfc7e47e04585b584913e39618731.wav
Audio loaded: 20240527131906_708cfc7e47e04585b584913e39618731.wav
[0.00s -> 1.12s]  What are you doing boxing?
you can see battery,glue,lead,object_1.  What are you doing boxing?
>>>I am not boxing. I am picking up the battery.
INFO:     2405:201:e00a:7103:6a35:286:baae:3cd3:0 - "POST /transcribe HTTP/1.1" 200 OK
FILE received: recorded_audio.wav
TEXT received: battery,glue,lead,object_1
FILE SAVED: 20240527132259_3ad878df74c